In [1]:
import pandas as pd
import numpy as np # linear algebra
from sklearn.preprocessing import LabelEncoder
# Replace 'path_to_dataset' with the actual path to your dataset file
df = pd.read_csv('embedding_FirstStructureL12.csv')
X_final= df

df2 = pd.read_excel('Dataset2neww.xlsx')
y = df2['Course']

label_encoder = LabelEncoder()


# Initialize LabelEncoder for the target variable y
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_encoded, test_size=0.2, random_state=42)

In [3]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

smote = SMOTE(random_state=42)
X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train, y_train)

borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled_borderline, y_train_resampled_borderline = borderline_smote.fit_resample(X_train, y_train)


## BorutaShap SMOTE + Bordeline SMOTE
focus on Bordeline SMOTE only also can

RF

In [4]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize Borderline SMOTE

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with Borderline SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with Borderline SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy (Best Random Forest with Borderline SMOTE): 0.3096045197740113
Classification Report (Best Random Forest with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.439     0.735     0.549        34
           2      0.333     0.500     0.400        32
           3      0.275     0.440     0.338        50
           4      0.128     0.154     0.140        39
           5      0.150     0.088     0.111        68
           6      0.188     0.182     0.185        33
           7      0.154     0.103     0.123        39
           8      0.254     0.250     0.252        64
           9      0.231     0.240     0.235        75
          10      0.154     0.113     0.130        53
          11      0.599     0.577     0.588       163
          12      0.062     0.043     0.051        23
          13      0.062     0.050     0.056   

In [5]:
import joblib

# Save the model to a file
joblib_file = "best_rf.joblib"
joblib.dump(best_rf_classifier, joblib_file)

['best_rf.joblib']

Gaussian NB

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


d:\python\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Parameters: {'var_smoothing': 1e-09}
Accuracy (Best GaussianNB with SMOTE): 0.21016949152542372
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.299     0.676     0.414        34
           2      0.213     0.406     0.280        32
           3      0.217     0.460     0.295        50
           4      0.048     0.026     0.033        39
           5      0.000     0.000     0.000        68
           6      0.077     0.091     0.083        33
           7      0.000     0.000     0.000        39
           8      0.174     0.188     0.180        64
           9      0.211     0.107     0.142        75
          10      0.067     0.113     0.085        53
          11      0.481     0.399     0.436       163
          12      0.107     0.130     0.118        23
          13      0.000     0.000     0.000        40
          14      0.391     0.115  

In [7]:
# Save the model to a file
joblib_file = "best_gnb.joblib"
joblib.dump(best_gnb_classifier, joblib_file)

['best_gnb.joblib']

DT

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.21016949152542372
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.500     0.529     0.514        34
           2      0.258     0.250     0.254        32
           3      0.149     0.200     0.171        50
           4      0.096     0.128     0.110        39
           5      0.065     0.074     0.069        68
           6      0.094     0.091     0.092        33
           7      0.031     0.026     0.028        39
           8      0.176     0.188     0.182        64
           9      0.176     0.160     0.168        75
          10      0.106     0.132     0.118        53
          11      0.534     0.429     0.476       163
          12      0.125     0.087     0.103        23
          13      0.117     

In [9]:
# Save the model to a file
joblib_file = "best_dt.joblib"
joblib.dump(best_dt_classifier, joblib_file)

['best_dt.joblib']

KNN


In [10]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Borderline SMOTE to the training data only
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with Borderline SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with Borderline SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with Borderline SMOTE): 0.25649717514124293
Classification Report (Best KNN with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.522     0.706     0.600        34
           2      0.394     0.406     0.400        32
           3      0.211     0.300     0.248        50
           4      0.150     0.231     0.182        39
           5      0.157     0.162     0.159        68
           6      0.097     0.091     0.094        33
           7      0.140     0.154     0.146        39
           8      0.211     0.188     0.198        64
           9      0.181     0.200     0.190        75
          10      0.080     0.075     0.078        53
          11      0.644     0.399     0.492       163
          12      0.250     0.174     0.205        23
          13      0.088    

In [11]:
# Save the model to a file
joblib_file = "best_knn.joblib"
joblib.dump(best_knn_classifier, joblib_file)

['best_knn.joblib']

SVM

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with BorderlineSMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with BorderlineSMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Parameters (SVM with BorderlineSMOTE): {'C': 100, 'gamma': 0.01}
Accuracy (SVM with BorderlineSMOTE): 0.3344632768361582
Classification Report (SVM with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.553     0.618     0.583        34
           2      0.452     0.438     0.444        32
           3      0.346     0.360     0.353        50
           4      0.189     0.179     0.184        39
           5      0.219     0.206     0.212        68
           6      0.200     0.152     0.172        33
           7      0.231     0.077     0.115        39
           8      0.176     0.234     0.201        64
           9      0.209     0.253     0.229        75
          10      0.143     0.094     0.114        53
          11      0.471     0.736     0.574       163
          12      0.200     0.043     0.071        23
          13      0.103     0.100     0.101        40
          14

In [13]:
# Save the model to a file
joblib_file = "best_svm.joblib"
joblib.dump(best_svm_classifier, joblib_file)

['best_svm.joblib']

LR

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with BorderlineSMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with BorderlineSMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with BorderlineSMOTE): {'C': 100, 'solver': 'liblinear'}
Accuracy (Logistic Regression with BorderlineSMOTE): 0.3593220338983051
Classification Report (Logistic Regression with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.512     0.647     0.571        34
           2      0.583     0.656     0.618        32
           3      0.429     0.420     0.424        50
           4      0.182     0.205     0.193        39
           5      0.303     0.338     0.319        68
           6      0.229     0.333     0.272        33
           7      0.184     0.179     0.182        39
           8      0.186     0.203     0.194        64
           9      0.250     0.280     0.264        75
          10      0.163     0.151     0.157        53
          11      0.750     0.607     0.671       163
    

In [15]:
joblib_file = "best_lr.joblib"
joblib.dump(best_lr_classifier, joblib_file)

['best_lr.joblib']

XGBoost

In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': None}
Accuracy (XGBoost with SMOTE): 0.32655367231638416
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.480     0.706     0.571        34
           2      0.375     0.469     0.417        32
           3      0.375     0.420     0.396        50
           4      0.140     0.154     0.146        39
           5      0.172     0.147     0.159        68
           6      0.200     0.182     0.190        33
           7      0.125     0.051     0.073        39
           8      0.230     0.266     0.246        64
           9      0.190     0.213     0.201        75
          10      0.167     0.151     0.158        53
          11      0.617     0.663     0.639       163
          12      0.188     0.130     0.154        23
          13

In [17]:
# Save the model to a file
joblib_file = "best_xgb.joblib"
joblib.dump(best_xgb_classifier, joblib_file)

['best_xgb.joblib']

AdaBoost

In [4]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 200}
Accuracy (AdaBoost with SMOTE): 0.1977401129943503
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.000     0.000     0.000         5
           1      0.600     0.529     0.562        34
           2      0.273     0.281     0.277        32
           3      0.244     0.380     0.297        50
           4      0.038     0.051     0.043        39
           5      0.048     0.029     0.036        68
           6      0.062     0.061     0.062        33
           7      0.129     0.103     0.114        39
           8      0.156     0.219     0.182        64
           9      0.179     0.200     0.189        75
          10      0.021     0.019     0.020        53
          11      0.512     0.252     0.337       163
          12      0.127     0.304     0.179        23
  

In [5]:
import joblib
# Save the model to a file
joblib_file = "best_ada.joblib"
joblib.dump(best_ada_classifier, joblib_file)

['best_ada.joblib']

CatBoost

LGBM

In [6]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018446 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97920
[LightGBM] [Info] Number of data points in the train set: 9655, number of used features: 384
[LightGBM] [Info] Start training from score -7.229321
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314
[LightGBM] [Info] Start training from score -2.773314


In [7]:
joblib_file = "best_lgbm_classifierMpnet.joblib"
joblib.dump(best_lgbm_classifier, joblib_file)

['best_lgbm_classifierMpnet.joblib']